In [7]:
from PIL import Image
import time
import cv2
import mss
import numpy as np
import imutils

In [8]:
img_path = r'D:\zother\auto-audition\data\8.png'
PLAY_AREA = (330, 510, 700, 580)
KEYS_AREA = (330, 540, 700, 580)

In [9]:
def show(cv_img):
    Image.fromarray(cv_img).show()

In [10]:
img = Image.open(img_path)
keys_img = img.crop(KEYS_AREA)
keys_img.show()

In [11]:
im = cv2.imread(img_path)
im = im[KEYS_AREA[1]:KEYS_AREA[3], KEYS_AREA[0]:KEYS_AREA[2]]
gray = cv2.cvtColor(im, cv2.COLOR_BGRA2GRAY)
show(im)

In [12]:
_, thres = cv2.threshold(gray, 254, 255, cv2.THRESH_BINARY)
show(thres)

In [31]:
def sort_contours(cnts):
	i = 0
	boundingBoxes = [cv2.boundingRect(c) for c in cnts]
	(cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
		key=lambda b:b[1][i]))
 
	# return the list of sorted contours and bounding boxes
	return (cnts, boundingBoxes)

def draw_contour(image, c, i):
	M = cv2.moments(c)
	cX = int(M["m10"] / M["m00"])
	cY = int(M["m01"] / M["m00"])
 
	cv2.putText(image, "#{}".format(i + 1), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
		0.5, (0, 255, 0), 1)
 
	return image

def get_key_img(thres_img, cnt):
	mask = np.zeros(thres_img.shape, dtype='uint8')
	cv2.drawContours(mask, [cnt], -1, 255, -1)
	res = cv2.bitwise_and(thres_img, thres_img, mask=mask)
	return res

def get_key_roi(thres_img, boxes):
	res = thres_img[boxes[1]:(boxes[1]+boxes[3]), boxes[0]:(boxes[0]+boxes[2])]
	return res

In [14]:
cnts = cv2.findContours(thres.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
orig = im.copy()

cv2.drawContours(orig, cnts, -1, (255, 0, 0), 1)
show(orig)

(cnts, boundingBoxes) = sort_contours(cnts)

sorted_cnts = im.copy()
for (i, c) in enumerate(cnts):
	draw_contour(sorted_cnts, c, i)

show(sorted_cnts)

In [28]:
key0_img = get_key_img(thres, cnts[0])
show(key0_img)

In [33]:
key0_roi = get_key_roi(thres, boundingBoxes[0])
show(key0_roi)

In [67]:
def get_dir(roi):
    h, w = roi.shape
    reg1 = roi[:, 0:w//3]
    reg2 = roi[2*h//3:h, :]
    reg3 = roi[:, 2*w//3:w]
    reg4 = roi[0:h//3, :]

    rate1 = cv2.countNonZero(reg1)
    rate2 = cv2.countNonZero(reg2)
    rate3 = cv2.countNonZero(reg3)
    rate4 = cv2.countNonZero(reg4)

    # show(reg1)
    # show(reg2)
    # show(reg3)
    # show(reg4)
    arr = np.array((rate1, rate2, rate3, rate4))
    # print(arr)

    direction = np.argmax(arr)
    return direction

In [68]:
for i in range(len(boundingBoxes)):
    key_roi = get_key_roi(thres, boundingBoxes[i])
    direction = get_dir(key_roi)
    print(direction+1)

1
2
3
2
4
1
2
3
4


In [58]:
show(thres)

In [59]:
thres.shape

(40, 370)

In [60]:
thres[:,0:10].shape

(40, 10)